In [387]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay

In [388]:
my_query = """ 
SELECT [PROJECT_SIZE_NAME] as 'Project_Name'
      ,[typeName] as 'Phase'
      ,[no]
      ,[categoryName] as 'Category'
      ,[riskType]
      ,[result]
      ,[rejectReason]
      ,[kickOffMeetingDate]
      ,[baseOverDueDate] as 'closeMeetingDate'
      ,[submitDate] as 'GMLASubmitDate'
  FROM [PR_WEB2].[dbo].[T_GMLA_CKLIST]

  LEFT JOIN [PR_WEB2].[dbo].[T_PS_ProdSizeRelateInfo]
  on projectSizeId = PROJECT_SIZE_ID
  /* Join project names */
  
  LEFT JOIN [PR_WEB2].[dbo].[T_GMLA_BS_VERSION_TYPE]
  ON typeId = [PR_WEB2].[dbo].[T_GMLA_BS_VERSION_TYPE].id
  /* Join GMLA#1/#2/#3 name*/
  
  LEFT JOIN [PR_WEB2].[dbo].[T_GMLA_CKLIST_CONFIRM]
  ON ([T_GMLA_CKLIST].projectSizeId = [T_GMLA_CKLIST_CONFIRM].projectSizeId 
  and [T_GMLA_CKLIST].typeId = [T_GMLA_CKLIST_CONFIRM].typeId)
  /* Join close meeting date and GMLA submit date */

  LEFT JOIN [PR_WEB2].[dbo].[T_GMLA_BS_CATEGORY]
  ON [T_GMLA_CKLIST].categoryId=[T_GMLA_BS_CATEGORY].id
  /* Join 6 category name via id */

  WHERE [result] != '0'
  /* Select completed GMLA */;
"""


In [389]:
#query by Unikey
server = 'T1-PE-SUPPORT' 
database = 'PR_WEB2' 
username = 'pieng' 
password = 'Q2iT5cwHJW3FH' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = my_query
df = pd.read_sql(query, cnxn)

In [390]:
df.columns

Index(['Project_Name', 'Phase', 'no', 'Category', 'riskType', 'result',
       'rejectReason', 'kickOffMeetingDate', 'closeMeetingDate',
       'GMLASubmitDate'],
      dtype='object')

In [391]:
df.dropna(axis=0, subset=['GMLASubmitDate'], inplace=True)

In [392]:
df.closeMeetingDate = pd.to_datetime(df.closeMeetingDate, utc=True)
df.GMLASubmitDate = pd.to_datetime(df.GMLASubmitDate,utc=True)

In [393]:
df.closeMeetingDate = df.closeMeetingDate.dt.tz_convert('Asia/Taipei')
df.GMLASubmitDate = df.GMLASubmitDate.dt.tz_convert('Asia/Taipei')

In [394]:
df['target_GMLASubmitDate'] = (df.closeMeetingDate + BDay(9))

In [395]:
df['on_time'] = df.target_GMLASubmitDate.dt.strftime('%Y-%m-%d') >= df.GMLASubmitDate.dt.strftime('%Y-%m-%d')

In [396]:
on_time = df.drop_duplicates(subset=['Project_Name','Phase']).sort_values(['Project_Name','Phase'])

In [397]:
on_time = on_time[on_time.target_GMLASubmitDate>'2021-01']

In [398]:
on_time[(on_time.on_time == False)].sort_values('target_GMLASubmitDate')

,Project_Name,Phase,no,Category,riskType,result,rejectReason,kickOffMeetingDate,closeMeetingDate,GMLASubmitDate,target_GMLASubmitDate,on_time
2735,fenix 7X,GMLA #2,2.A-1,Product development,1,3,None,None,2020-12-30 00:00:00+08:00,2021-01-15 14:36:26.085105100+08:00,2021-01-12 00:00:00+08:00,False
56,DGCN20,GMLA #3,3.A-5,Product development,1,3,3.C-6: please find out what's difference of CP...,None,2021-01-26 00:00:00+08:00,2021-02-22 17:34:40.150389+08:00,2021-02-08 00:00:00+08:00,False
1705,Edge 1040,GMLA #1,1.A-1,Product development,1,3,None,None,2021-02-05 00:00:00+08:00,2021-02-19 17:29:48.395736400+08:00,2021-02-18 00:00:00+08:00,False
1252,One Battery Bike Project,GMLA #3,3.A-5,Product development,1,3,Please help to modify 3.C-5 issue description.,None,2021-02-05 00:00:00+08:00,2021-03-04 15:19:42.207827800+08:00,2021-02-18 00:00:00+08:00,False
3021,Descent G1,GMLA #1,1.A-1,Product development,1,3,Strain gauge need to update the V3 PCB result ...,None,2021-03-17 00:00:00+08:00,2021-04-03 08:24:22.410314400+08:00,2021-03-30 00:00:00+08:00,False
2780,fenix 7X,GMLA #3,3.A-5,Product development,1,3,Revise 3.A-7 content (some 007- do not under 1...,2021-03-07 16:00:00.0000000 +00:00,2021-05-10 00:00:00+08:00,2021-05-24 11:20:06.622288900+08:00,2021-05-21 00:00:00+08:00,False
5240,HRM-Pro Plus,GMLA #1,1.A-1,Product development,1,3,None,None,2021-05-21 00:00:00+08:00,2021-12-20 17:25:34.375031600+08:00,2021-06-03 00:00:00+08:00,False
2552,Varia Radar Camera (RCT715),GMLA #2,2.A-1,Product development,1,3,update the mitigation plan of fail items,2021-07-04 16:00:00.0000000 +00:00,2021-08-30 00:00:00+08:00,2021-09-23 16:20:34.945172800+08:00,2021-09-10 00:00:00+08:00,False
3566,"vivomove ""Lite""",GMLA #3,3.A-5,Product development,1,3,some items are under confrimed,2021-06-30 16:00:00.0000000 +00:00,2021-09-13 00:00:00+08:00,2021-09-27 22:15:16.189070600+08:00,2021-09-24 00:00:00+08:00,False
1045,BMW MGU22-RSE,GMLA #3,3.A-5,Product development,1,3,"3.E-4, 3.E-12 項目請修正。",2021-07-19 16:00:00.0000000 +00:00,2021-09-24 00:00:00+08:00,2021-10-19 13:51:42.345835500+08:00,2021-10-07 00:00:00+08:00,False


In [399]:
on_time_2021 =on_time.set_index('target_GMLASubmitDate').resample(rule='m').agg({'Project_Name':'count','on_time':'sum','rejectReason':'count'})

In [400]:
on_time_2021.reset_index(inplace=True)

In [401]:
on_time_2021['target_GMLASubmitDate'] = on_time_2021['target_GMLASubmitDate'].dt.strftime('%Y-%m')

In [402]:
on_time_2021.set_index('target_GMLASubmitDate')

,Project_Name,on_time,rejectReason
target_GMLASubmitDate,,,
2021-01,9,8,2
2021-02,9,6,3
2021-03,10,9,1
2021-04,14,14,4
2021-05,10,9,1
2021-06,14,13,0
2021-07,8,8,3
2021-08,2,2,1
2021-09,9,7,2
